In [11]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import sys
from os import listdir

In [12]:
def create_paper_summary_data_frame():
    doi = input("DOI for paper: ")
    directory = input("Directory for paper: ")
    
    #list of all the [folders/plots] for paper ()
    plots_in_dir = !ls $directory 
    num_plots = len(plots_in_dir)
    
    for i in range(num_plots):
        paper_descript = input("Describe some key aspects about the paper ")
        plot_foldername = directory + "/" + plots_in_dir[i]
        
        # list of all the csv's associated with plot
        data_segs_for_plot = !ls $plot_foldername 
        print(data_segs_for_plot)
        num_segs_in_plot = len(data_segs_for_plot)
        x_label = widgets.Dropdown(
            options={'Air Vel [cm/s]': 1,'FSR [mm/s]': 2,'d_c [mm]': 3, 'd_i [mm]': 4, 'P_atm': 5, '[O2]': 6, 'g': 7, 
                     'q_ext [kW/m^2]': 8, 'core material': 9, 'insulation material': 10,'orientation (vert/horiz)': 11, 
                     'up | down': 12},
            value=1,
            description='X - Axis Label:',
            disabled=False,
        )
        display(x_label)
        y_label = widgets.Dropdown(
            options={'Air Vel [cm/s]': 1,'FSR [mm/s]': 2,'d_c [mm]': 3, 'd_i [mm]': 4, 'P_atm': 5, '[O2]': 6, 'g': 7, 
                     'q_ext [kW/m^2]': 8, 'core material': 9, 'insulation material': 10,'orientation (vert/horiz)': 11, 
                     'up | down': 12},
            value=2,
            description='Y - Axis Label:',
            disabled=False,
        )
        display(y_label)
        
        cols = ['Air Vel [cm/s]','FSR [mm/s]','d_c [mm]', 'd_i [mm]',
                      'P_atm', '[O2]', 'g', 'q_ext [kW/m^2]', 'core material', 
                      'insulation material','orientation (vert/horiz)', 'up | down']
        final_df = pd.DataFrame(columns = cols)
        
        for j in range(num_segs_in_plot):
            name_of_seg_in_plot = plot_foldername + "/" + data_segs_for_plot[j]
            print(name_of_seg_in_plot)
            skeleton_row = []
            for k in cols:
                if k in ['core material', 'insulation material', 'orientation (vert/horiz)', 'up | down']:
                    skeleton_row.append(str(input("For data seg " + data_segs_for_plot[j] + " " + k + " ")) or 'NA')
                else: 
                    skeleton_row.append(float(input("For data seg " + data_segs_for_plot[j] + " " + k + " ") or 'nan'))
            skeleton_df = pd.DataFrame(np.array([skeleton_row]), columns=cols)
            print(skeleton_df)
            final_df.append(skeleton_df)
            print(final_df)
        print(final_df)

In [ ]:
create_paper_summary_data_frame()


DOI for paper: 3
Directory for paper: kobayashi
Describe some key aspects about the paper 3
['8mm_Cu.csv', '8mm_SS.csv', '9mm_Cu.csv', '9mm_SS.csv']


Dropdown(description='X - Axis Label:', options={'Air Vel [cm/s]': 1, 'FSR [mm/s]': 2, 'd_c [mm]': 3, 'd_i [mm…

Dropdown(description='Y - Axis Label:', index=1, options={'Air Vel [cm/s]': 1, 'FSR [mm/s]': 2, 'd_c [mm]': 3,…

kobayashi/1/8mm_Cu.csv
For data seg 8mm_Cu.csv Air Vel [cm/s] 3
For data seg 8mm_Cu.csv FSR [mm/s] 3
For data seg 8mm_Cu.csv d_c [mm] 3
For data seg 8mm_Cu.csv d_i [mm] 3
For data seg 8mm_Cu.csv P_atm 3
For data seg 8mm_Cu.csv [O2] 3
For data seg 8mm_Cu.csv g 3
For data seg 8mm_Cu.csv q_ext [kW/m^2] 3
For data seg 8mm_Cu.csv core material 3
For data seg 8mm_Cu.csv insulation material 3
For data seg 8mm_Cu.csv orientation (vert/horiz) 3
For data seg 8mm_Cu.csv up | down 3
  Air Vel [cm/s] FSR [mm/s] d_c [mm] d_i [mm] P_atm [O2]    g q_ext [kW/m^2]  \
0            3.0        3.0      3.0      3.0   3.0  3.0  3.0            3.0   

  core material insulation material orientation (vert/horiz) up | down  
0             3                   3                        3         3  
Empty DataFrame
Columns: [Air Vel [cm/s], FSR [mm/s], d_c [mm], d_i [mm], P_atm, [O2], g, q_ext [kW/m^2], core material, insulation material, orientation (vert/horiz), up | down]
Index: []
kobayashi/1/8mm_SS.csv


3### 3
3### df_read1 = pd.read_csv('kobayashi\8mm_Cu.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read2 = pd.read_csv('kobayashi\9mm_Cu.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read3 = pd.read_csv('kobayashi\8mm_SS.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read4 = pd.read_csv('kobayashi\9mm_SS.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read5 = pd.read_csv
df_togeter = pd.concat([df_read1, df_read2])
print(df_togeter)

In [15]:
'asd' in ['asd', 'dsa']

True

In [28]:
"" or False

False

In [31]:
int('NA')

ValueError: invalid literal for int() with base 10: 'NA'